In [ ]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
import math
import random

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (6,6)

<h1>More Regression</h1>

We can now generate linear regression models that can predict a Y value when we supply an X value - cool!

What about accuracy. How much can we trust these models to make accurate predictions?

For regression we have two primary metrics of accuracy that we look at first - <b>RMSE and R2 (R squared)</b>. We can look at each individually, they are both pretty simple and easy to calculate. 

We'll load some data where there is more error than the NBA/NFL example from last time. 

In [ ]:
#Load Data
df = pd.read_csv("data/WalMartSales.csv")
df.head()

We can look at how accurately can we predict store sales at a WalMart, if we know the size of the store. 

In [ ]:
#We can take a preview
x1 = "StoreSize"
y1 = "Sales"
d = df[["StoreSize", "Sales"]]
sns.pairplot(d, kind="reg")

It appears that there is a relationship, though not a super strong one. Regression time...

In [ ]:
#Do regression
inter, slope = thinkstats2.LeastSquares(d[x1],d[y1])
res = thinkstats2.Residuals(d[x1], d[y1], inter, slope)
regLine = thinkstats2.FitLine(d[x1], inter, slope)
print("Y intercept:", inter)
print("Slope:", slope)

In [ ]:
#Plot with regression line
sns.scatterplot(x=d[x1], y=d[y1])
sns.lineplot(x=regLine[0], y=regLine[1])

As a side observation, it looks like WalMart makes a lot of stores that are the same size. Someone might think about transforming the store size into a categorical value - for example, if each size of store is kind of a different "type", it might make sense. This type of decision would probably require both some knowledge of how the stores are setup and some trials to observe the results (this would presumably be when we were making a model with many inputs). 

In [ ]:
#Plot Residuals
sns.residplot(x=d[x1], y=d[y1])

### Results and Error

Ok. We have a regression, it doesn't really look like it is the best one. 

We need to quantify the accuracy. First - RMSE. 

The RMSE is the same concept of every other time we've looked at it - take the residuals, square them, sum those up, take the square root. We can build a pretty simple calculator to find RMSE, or we can use one of the library functions. 

In [ ]:
#Calculate RMSE from residuals
def rmseFromResiduals(residuals):
    SSE = 0
    for i in range(len(residuals)):
        SSE += (residuals[i]**2)
    MSE = SSE/len(residuals)
    return math.sqrt(MSE)

In [ ]:
rmseFromResiduals(res)

### RMSE Results from Models

What if we are using one of the library functions? We'll use statsmodels here. The RMSE stuff is the same for sklearn. We make the predictions from the model, then compare those predictions to the real y values via one of the error calculating functions. 

In [ ]:
#Fit the model. 
import statsmodels.api as sm
X2 = sm.add_constant(d[x1])
est = sm.OLS(d[y1], X2)
est2 = est.fit()
print(est2.summary())

Now to generate an RMSE, we need to get the actual predictions. 

In [ ]:
#Fit the model which we have already done
#Now generate predictions for our data
ypred = est2.predict(X2)

At this point, we have the predictions, we also have the real values, we can use any RMSE caclulator to get the answer. 

In [ ]:
#Calculate the RMSE, here using the statsmodel function. 
from statsmodels.tools.eval_measures import rmse
rmse(d[y1], ypred)

In [ ]:
#Sklearn like we did way back before. 
mean_squared_error(d[y1], ypred, squared=False)

In [ ]:
#Or define a function to use what we made.
def rmseFromPredictions(real, predicted):
    return rmseFromResiduals(real-predicted)

In [ ]:
rmseFromPredictions(d[y1], ypred)

#### Results

RMSE is a major metric for the accuracy of our model, it tells us how close our predictions are. 

## Exercise

Now, you try one! Load some different data and get the RMSE. Try to do it two ways, one with a library function, and one with the RMSE calculation we made. 

Use the Knee value as the X and Bodyfat as the Y. Build a model to predict bodyfat from knee size. Calculate the RMSE for that model.

In [ ]:
df_exercise = pd.read_csv("data/bodyfat.csv")
df_exercise.head()

In [ ]:
sns.regplot(data=df_exercise, x="Knee", y="BodyFat")

In [ ]:
#Work exercise
#Do regression


## R2 - Coefficient of Determination

One other metric that is important in evaluating our model is called R2 or R2 or the Coefficient of Determination. The definition of R2 is "the percentage of variation in the dependent or predicted variable (y) that can be explained by variation in the independent or explanatory variable (x)" - or roughly how much of the total picture our model captures. R2 values go up from 0 to 1 (you can get other values, but that's an indication the model is pretty useless. Only 0 to 1 matters in practice), and they are expressable as a percent. The R2 value is also called the "goodness of fit". 

For an example of what R2 means, think about a car stopping via its brakes. The R2 of brakes vs stopping distance might be .8 - or 80% of how long it takes you to stop is due to how hard you press the brake pedal. The other 20% is other stuff - tire grip on the road, tail/head wind, ABS modulation, etc... For the walmart one we are doing here, the R2 will tell us what percentage of sales are attributed to store size, and what percentage is due to other stuff - location, prices, products, staff, coupons, etc...

We can calculate this pretty simply - it is the ratio of the variance of the residuals to the varaiance of the y values in total. 

![R Squared](images/r_squared_form.jpg "R Squared")

We can think of it intuitively:
<ul>
<li> The data varries. Literally, the data points just have different values. 
<li> Our model captures <i>some</i> of that variation - specifically the part that <i>isn't</i> the residual. 
    <ul>
    <li> Recall - the residual is how <b>wrong</b> our model is, the "other part" is how <b>right</b> our model is, or the sum of squares of the regression. 
    </ul>
<li> The greater a percentage of the total variation is "in" our model, the better it fits - or the closer the pattern of the model's predictions are tailored to the real data. 
<li> The lower a percentage of the total variation is outside of our model, in our residuals, the less closely the model follows the data. 
<li><b> This percentage is R Sqaured - the coefficient of determination.</b>
</ul>

![R Squared](images/r_squared.jpg "R Squared")

### What Does R2 Tell Us?

R sqaured isn't directly a measure of error, though it is derived from the errors. R2 is a measure of how well suited the model we are using is for the data we are using it with (later in the machine learning stuff we'll look at visualizations of models that illustrate this a bit more clearly, we need some other stuff to get to that point though). High R2 tell us we are probably using a suitable model, low R2 tells us that we may want to make changess. It is also always on the same scale, so we can compare between models pretty easily. 

R2 doesn't tell us our accuracy like the RMSE does, as we don't get a direct measure of the error in predictions. Since they are both based on the error though, they will almost always move in concert with each other - as the error amounts drop, the R2 rises, and vice versa. 

<b>Note:</b> R squared is on a 0 to 1 scale. Sometimes calculations spit out an R2 that is negative. If this happens we don't really care about the value, it is an indication that the model is a terrible fit for the data, worse than if you were to just use a constant horizontal line as a model. E.g.:

![Negative R Squared](images/neg_r_squared.png "Negative R Squared")

In [ ]:
def rSquared(yvals, residuals):
    return (1-(thinkstats2.Var(residuals)/thinkstats2.Var(yvals)))

It can also be expressed like this, this is the definition you get from Wikipedia. 1 - SumSquaresResiduals/TotalSumSquares

In [ ]:
def rSquared_2(yvals, residuals):
    RSS = 0
    TSS = 0
    ybar = yvals.mean()
    for i in range(len(residuals)):
        RSS += (residuals[i]**2)
        TSS += ((yvals[i]-ybar)**2)
    r2 = 1-(RSS/TSS)
    return r2

In [ ]:
rSquared_2(d[y1], res)

<b>As a fun note, this is equal to the Pearson correlation squared. </b>

In [ ]:
1-thinkstats2.Corr(d[y1], res)**2

Hopefully those two match what the statsmodels has above!!!!

#### Using Errors

These two values - RMSE and R2 - are the two primary (mostly - more on this later, along with R2 limitations) metrics that we can use to evaluate a model's accuracy and its predictive value. If we have a high R2 and a (comparatively) low RMSE, we are probably making reliable predictions. 

<b>Big Note:</b> our real life error calculation process will be slightly different from this - notably rather than calculating the error amounts from the model itself, we will calculate it based on some other, test, data. The errors themselves are the same, but we will split our data to make the error caclulations more robust. We'll look at this next time. 

## Exercise

Calculate R2 for the bodyfat/knee example.

In [ ]:
#R2 for the knee/bodfat


## Exercise - End to End Regression

Now, load some other data and give it a try... 

Do a linear regression and calculate R2 and the RMSE for this data. Use head size to predict brain weight. It is pretty easy to measure the size of someone's dome, but pretty hard to weigh their brain. We want to see how accurately we can predict one from the other. Does the model look like we can trust it to make good predictions?  

In [ ]:
df2 = pd.read_csv("data/brain.csv")
df2.head()

<b>You are now a machine learning professional. </b>